# PODCAST SUMMARIZER USING PYTHON  SCRIPT

Importing libraries


In [5]:
#importing libraries
import pandas as pd
import transformers
import openai
from transformers import BartTokenizer, BartForConditionalGeneration
import os

In [ ]:


openai.api_key = "YOUR_OPENAI_API_KEY"

def summarize_podcast(podcast_transcript, max_tokens=4096):
    prompt = f"Summarize the following podcast transcript:\n\n{podcast_transcript}"

    response = openai.Completion.create(
        engine="text-davinci-002",  # GPT-3.5
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0.7,
    )

    return response.choices[0].text.strip()

# Example podcast transcript (replace this with your actual transcript)
podcast_transcript = """
Your long podcast transcript here...
"""

# Summarize the podcast
summary = summarize_podcast(podcast_transcript)
print(summary)


In [12]:
#specifying file path
file_path = r"116_diarized_timestamped_transcript.txt"


def summarize_podcast(file_path, max_input_length=1024, max_output_length=150):
    # Load the podcast transcript from the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        podcast_transcript = file.read()

    # Tokenize and truncate the input text
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
    inputs = tokenizer(podcast_transcript, max_length=max_input_length, return_tensors="pt", truncation=True)

    # Load pre-trained BART model
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

    # Generate the summary
    summary_ids = model.generate(inputs.input_ids, max_length=max_output_length, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example usage
# file_path = "path/to/your/podcast_transcript.txt"
summary = summarize_podcast(file_path)
print(summary)


ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.